In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pathlib, os, subprocess, json, datetime, shutil

REPO_URL   = "https://github.com/SzymonSmagowski/DeepLearningCourse.git"
BRANCH     = "main"
DATA_IN_DRIVE = "data/speech_commands_v0.01"

ROOT_DRIVE   = pathlib.Path("/content/drive/MyDrive")
DATA_SRC     = ROOT_DRIVE / DATA_IN_DRIVE
DATA_DST     = pathlib.Path("/content/data")                        # where the script expects it
REPO_DIR     = pathlib.Path("/content") / pathlib.Path(REPO_URL).stem
LOGS_DRIVE   = ROOT_DRIVE / "speech_runs"      # all logs live here

print("Repo dir :", REPO_DIR)
print("Data src :", DATA_SRC)
print("Data dst :", DATA_DST)
print("Logs dir :", LOGS_DRIVE)

In [ ]:
if not REPO_DIR.exists():
    !git clone -b "$BRANCH" "$REPO_URL" "$REPO_DIR"
else:
    %cd $REPO_DIR
    !git pull origin "$BRANCH"
    %cd -

In [ ]:
# Fast option: symlink (needs Drive mounted read-only)
if DATA_DST.exists():
    print("✓ dataset already present")
else:
    DATA_DST.parent.mkdir(parents=True, exist_ok=True)
    try:
        DATA_DST.symlink_to(DATA_SRC, target_is_directory=True)
        print("Created symlink → much faster")
    except Exception as e:
        print("Symlink failed, falling back to copy (may take 2-3 min)…")
        shutil.copytree(DATA_SRC, DATA_DST)

In [ ]:
!pip install -q torch torchaudio librosa tqdm matplotlib seaborn scikit-learn